In [2]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

train_path = 'C:/Users/Pratik/OneDrive/Desktop/pr_project/base_dir/train_dir'
val_path = 'C:/Users/Pratik/OneDrive/Desktop/pr_project/base_dir/val_dir'


# Initialize lists for train data
train_images = []
train_labels = []

# Cap number of images per label
max_images_per_label = 4410

# Process training directory
for label in os.listdir(train_path):
    label_count = 0
    for image in os.listdir(os.path.join(train_path, label)):
        if label_count >= max_images_per_label:
            break
        filepath = os.path.join(train_path, label, image)
        train_images.append(filepath)
        train_labels.append(label)
        label_count += 1

# Convert training data to DataFrame
train_df = pd.DataFrame({
    'Images': train_images,
    'Labels': train_labels
})

# Initialize lists for validation data
val_images = []
val_labels = []

# Process validation directory
for label in os.listdir(val_path):
    for image in os.listdir(os.path.join(val_path, label)):
        filepath = os.path.join(val_path, label, image)
        val_images.append(filepath)
        val_labels.append(label)

# Convert validation data to DataFrame
val_df = pd.DataFrame({
    'Images': val_images,
    'Labels': val_labels
})

# Display DataFrame information
print("Training Data")
print(train_df.head())
print(train_df['Labels'].value_counts())

print("\nValidation Data")
print(val_df.head())
print(val_df['Labels'].value_counts())


ModuleNotFoundError: No module named 'numpy'

In [16]:
# Train/validation split from train_df (assuming train_df contains all your training data)
X_train, X_dummies = train_test_split(
    train_df, test_size=0.05, random_state=42)

# Test/validation split from X_dummies
X_test, X_val = train_test_split(X_dummies, test_size=0.5, random_state=42)

# Display the splits information
print("Training Data")
print(X_train.head())
print(X_train['Labels'].value_counts())

print("\nTest Data")
print(X_test.head())
print(X_test['Labels'].value_counts())

print("\nValidation Data")
print(X_val.head())
print(X_val['Labels'].value_counts())


Training Data
                                                  Images Labels
26808  C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...   vasc
29600  C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...   vasc
18900  C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...    mel
26653  C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...   vasc
27036  C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...   vasc
Labels
vasc     4210
akiec    4201
nv       4196
mel      4192
df       4183
bkl      4177
bcc      4167
Name: count, dtype: int64

Test Data
                                                  Images Labels
3858   C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...  akiec
6253   C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...    bcc
27099  C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...   vasc
6498   C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...    bcc
16085  C:/Users/Pratik/OneDrive/Desktop/pr_project/ba...     df
Labels
bcc      132
df       113
bkl      113
mel      111
nv       104
akie

In [17]:
# Initialize ImageDataGenerator
# Rescaling pixel values to [0, 1]
image_gen = ImageDataGenerator(rescale=1./255)

# Train data generator
train = image_gen.flow_from_dataframe(
    dataframe=X_train,
    x_col="Images",
    y_col="Labels",
    target_size=(224, 224),  # Adjusted target_size
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True  # Shuffle for training data
)

# Test data generator
test = image_gen.flow_from_dataframe(
    dataframe=X_test,
    x_col="Images",
    y_col="Labels",
    target_size=(224, 224),  # Adjusted target_size
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False  # Shuffle off for testing data
)

# Validation data generator
val = image_gen.flow_from_dataframe(
    dataframe=X_val,
    x_col="Images",
    y_col="Labels",
    target_size=(224, 224),  # Adjusted target_size
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False  # Shuffle off for validation data
)


Found 29326 validated image filenames belonging to 7 classes.
Found 772 validated image filenames belonging to 7 classes.
Found 772 validated image filenames belonging to 7 classes.


In [18]:
Classes = list(train.class_indices.keys())
print(Classes)


['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


In [19]:
from tensorflow.keras.layers import Dropout


In [20]:
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf

# Define the input shape
img_shape = (224, 224, 3)

# Load the Xception base model
base_model = tf.keras.applications.Xception(include_top=False, weights="imagenet",
                                            input_shape=img_shape, pooling='max')

# Freeze the base model layers
base_model.trainable = False

# Construct the classification head
model = Sequential([
    base_model,
    Flatten(),
    Dropout(rate=0.3),
    Dense(128, activation='relu'),
    Dropout(rate=0.25),
    Dense(7, activation='softmax')  # 7 units for the 7 classes
])

# Compile the model
model.compile(optimizer=Adamax(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 2048)           │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,124,655 (80.58 MB)

 Trainable params: 263,175 (1.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [21]:
history = model.fit(train, epochs=10, validation_data=val, verbose=1)


c:\Users\Pratik\OneDrive\Desktop\pr_project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1619s 4s/step - accuracy: 0.4114 - loss: 1.5894 - val_accuracy: 0.5907 - val_loss: 1.0337
Epoch 2/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 2124s 5s/step - accuracy: 0.5590 - loss: 1.1688 - val_accuracy: 0.6412 - val_loss: 0.9400
Epoch 3/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1218s 3s/step - accuracy: 0.5904 - loss: 1.0781 - val_accuracy: 0.6503 - val_loss: 0.9058
Epoch 4/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1148s 2s/step - accuracy: 0.6095 - loss: 1.0263 - val_accuracy: 0.6723 - val_loss: 0.8423
Epoch 5/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1136s 2s/step - accuracy: 0.6295 - loss: 0.9787 - val_accuracy: 0.6839 - val_loss: 0.8343
Epoch 6/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1134s 2s/step - accuracy: 0.6375 - loss: 0.9474 - val_accuracy: 0.6865 - val_loss: 0.7912
Epoch 7/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1132s 2s/step - accuracy: 0.6554 - loss: 0.9077 - val_accuracy: 0.7163 - val_loss: 0.7786
Epoch 8/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1131s 2s/step - accuracy: 0.6603 - loss: 0.8948 - 

In [22]:
model.evaluate(test, verbose=1)


13/13 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.7108 - loss: 0.7986


[0.7869207262992859, 0.7020725607872009]

In [24]:
# Save the trained model in .keras format
model.save('saved_model.keras')


In [25]:
from tensorflow.keras.models import load_model
import tensorflow as tf

# Step 1: Load the saved model
model = load_model('saved_model.keras')

# Step 2: Unfreeze the last few layers of the base model (Xception)
# Assuming the base_model is the first layer in the saved model
base_model = model.layers[0]
base_model.trainable = True

# Unfreeze all layers from a specific layer forward, if needed (optional)
# For example, to unfreeze only the top 20 layers of the base model:
# for layer in base_model.layers[:-20]:
#     layer.trainable = False

# Step 3: Re-compile the model with a lower learning rate for fine-tuning
model.compile(
    # Use a smaller learning rate for fine-tuning
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 4: Continue training the model
history = model.fit(train, epochs=10, validation_data=val, verbose=1)


Epoch 1/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 5680s 12s/step - accuracy: 0.5921 - loss: 1.0859 - val_accuracy: 0.8899 - val_loss: 0.2844
Epoch 2/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 5678s 12s/step - accuracy: 0.9187 - loss: 0.2390 - val_accuracy: 0.9262 - val_loss: 0.2084
Epoch 3/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 5655s 12s/step - accuracy: 0.9695 - loss: 0.0892 - val_accuracy: 0.9430 - val_loss: 0.1885
Epoch 4/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 5665s 12s/step - accuracy: 0.9844 - loss: 0.0473 - val_accuracy: 0.9534 - val_loss: 0.1925
Epoch 5/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 5612s 12s/step - accuracy: 0.9897 - loss: 0.0336 - val_accuracy: 0.9482 - val_loss: 0.1934
Epoch 6/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 5585s 12s/step - accuracy: 0.9891 - loss: 0.0311 - val_accuracy: 0.9275 - val_loss: 0.2848
Epoch 7/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 5588s 12s/step - accuracy: 0.9916 - loss: 0.0267 - val_accuracy: 0.9521 - val_loss: 0.2389
Epoch 8/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 5590s 12s/step - accuracy: 0.9926 - loss: 0

In [26]:
# Save the fine-tuned model
model.save('fine_tuned_model.keras')
print("Model saved as 'fine_tuned_model.keras'")


Model saved as 'fine_tuned_model.keras'


In [27]:
# Save the fine-tuned model
model.save('fine_tuned_model.h5')
print("Model saved as 'fine_tuned_model.h5'")


Model saved as 'fine_tuned_model.h5'


In [28]:
import pickle
from joblib import dump, load
import os

# Example metadata to save (replace with actual metadata like class mappings)
metadata = {
    'classes': list(train.class_indices.keys()),  # Class names
    'input_shape': (224, 224, 3),                # Model input shape
    'preprocessing': 'normalization',            # Description of preprocessing
}

# Save metadata with pickle
with open('model_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

# Save metadata with joblib
dump(metadata, 'model_metadata.joblib')

print("Metadata saved using pickle and joblib.")

# Load metadata back (if needed)
with open('model_metadata.pkl', 'rb') as f:
    metadata_pkl = pickle.load(f)
print("Loaded from pickle:", metadata_pkl)

metadata_joblib = load('model_metadata.joblib')
print("Loaded from joblib:", metadata_joblib)


Metadata saved using pickle and joblib.
Loaded from pickle: {'classes': ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'], 'input_shape': (224, 224, 3), 'preprocessing': 'normalization'}
Loaded from joblib: {'classes': ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'], 'input_shape': (224, 224, 3), 'preprocessing': 'normalization'}


In [29]:
class ModelPipeline:
    def __init__(self, model, metadata):
        self.model = model
        self.metadata = metadata


# Wrap the trained model and metadata into a custom object
pipeline = ModelPipeline(
    model=model,
    metadata={
        'classes': list(train.class_indices.keys()),  # Class names
        'input_shape': (224, 224, 3)                # Model input shape
    }
)

# Save the pipeline with pickle
with open('model_pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

# Save the pipeline with joblib
dump(pipeline, 'model_pipeline.joblib')
print("Model pipeline saved using pickle and joblib.")

# Load the pipeline back
with open('model_pipeline.pkl', 'rb') as f:
    loaded_pipeline = pickle.load(f)
print("Loaded pipeline with pickle.")

loaded_pipeline_joblib = load('model_pipeline.joblib')
print("Loaded pipeline with joblib.")

# You can now access the model and metadata
print("Pipeline Model Summary:")
loaded_pipeline.model.summary()
print("Classes:", loaded_pipeline.metadata['classes'])


Model pipeline saved using pickle and joblib.


c:\Users\Pratik\OneDrive\Desktop\pr_project\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 160 variables whereas the saved optimizer has 318 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Loaded pipeline with pickle.
Loaded pipeline with joblib.
Pipeline Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 2048)           │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,194,784 (160.96 MB)

 Trainable params: 21,070,127 (80.38 MB)

 Non-trainable params: 54,528 (213.00 KB)

 Optimizer params: 21,070,129 (80.38 MB)

Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
